In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import yfinance as yf
import json

In [2]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_income_stmt'
table_conca = f'{project}.{dataset}.{table}'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
unique_tickers = get_unique_tickers()

In [5]:
import yfinance as yf
import pandas as pd

data = []

for _, row in unique_tickers.iterrows():
    ticker = row['ticker']  # Corregir el nombre de la columna para acceder al ticker

    try:
        # Obtener datos históricos
        data_row = yf.Ticker(ticker).income_stmt

        # Agregar el ticker como una columna en los datos históricos
        data_row['ticker'] = ticker
        # Añadir los datos a la lista
        data.append(data_row)
        print(f"Datos obtenidos para {ticker}.")
    except Exception as e:
        print(f"Error al obtener datos para {ticker}: {e}")

# Concatenar todos los DataFrames en uno solo
df = pd.concat(data, ignore_index=True)
df

Datos obtenidos para AAPL.
Datos obtenidos para AMGN.
Datos obtenidos para AMZN.
Datos obtenidos para AXP.
Datos obtenidos para BA.
Datos obtenidos para CAT.
Datos obtenidos para CRM.
Datos obtenidos para CSCO.
Datos obtenidos para CVX.
Datos obtenidos para DIS.
Datos obtenidos para DOW.
Datos obtenidos para GS.
Datos obtenidos para HD.
Datos obtenidos para HON.
Datos obtenidos para IBM.
Datos obtenidos para INTC.
Datos obtenidos para JNJ.
Datos obtenidos para JPM.
Datos obtenidos para KO.
Datos obtenidos para MCD.
Datos obtenidos para MMM.
Datos obtenidos para MRK.
Datos obtenidos para MSFT.
Datos obtenidos para NKE.
Datos obtenidos para PG.
Datos obtenidos para TRV.
Datos obtenidos para UNH.
Datos obtenidos para V.
Datos obtenidos para VZ.
Datos obtenidos para WMT.


,2023-09-30 00:00:00,2022-09-30 00:00:00,2021-09-30 00:00:00,2020-09-30 00:00:00,ticker,2023-12-31 00:00:00,2022-12-31 00:00:00,2021-12-31 00:00:00,2020-12-31 00:00:00,2019-12-31 00:00:00,...,2019-09-30 00:00:00,2024-06-30 00:00:00,2023-06-30 00:00:00,2022-06-30 00:00:00,2021-06-30 00:00:00,2024-05-31 00:00:00,2023-05-31 00:00:00,2022-05-31 00:00:00,2021-05-31 00:00:00,2020-06-30 00:00:00
0,0.0,0.0,0.0,0.0,AAPL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.147,0.162,0.133,0.144,AAPL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,129188000000.0,133138000000.0,123136000000.0,81020000000.0,AAPL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,96995000000.0,99803000000.0,94680000000.0,57411000000.0,AAPL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11519000000.0,11104000000.0,11284000000.0,11056000000.0,AAPL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,NaN,NaN,NaN,NaN,WMT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1377,NaN,NaN,NaN,NaN,WMT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1378,NaN,NaN,NaN,NaN,WMT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1379,NaN,NaN,NaN,NaN,WMT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Asegúrate de que los nombres de las columnas sean strings y recórtalos
df.columns = [str(col)[:10] for col in df.columns]

In [7]:
# Identificar columnas de tipo 'object'
object_columns = df.select_dtypes(include=['object']).columns

# Convertir las columnas de tipo 'object' a listas
for col in object_columns:
    df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x)

In [8]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset, table, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
